# What happens:
File is created and email is prepared

## Import 

In [1]:
import os
import sys
import pandas as pd
import numpy as np
from pandas import DataFrame
import sqlalchemy
from sqlalchemy import create_engine
import getpass
import win32com.client as win32

from openpyxl import Workbook, load_workbook

### Select the folder where you wanna do the work
### Also ask for the Reporting Date

In [2]:
directory = os.path.abspath(os.getcwd()) + "/"
rd = str(input("Please enter the RD (YYYYMMDD): "))

ws = "0194"

### Make connection with Oracle DB

In [3]:
#Setting up RAY connection:
connection_RAY = create_engine(str("oracle://:@OCRPN"))
connection_RAY.execute("call pack_context.context_open(to_date('{reporting_date}','YYYYMMDD'),{partition})".format(
                 reporting_date = rd, partition = ws))

## Load template

In [4]:
im_folder = "Import/"
file_path = "equity_uncons_template_v2.xlsx"
equity_template = load_workbook(im_folder + file_path)
equity_template_sql = equity_template['sql']
equity_sql = equity_template_sql['A6'].value

### Save selection
Also see how much time it took

In [5]:
%%time
d = pd.read_sql_query(equity_sql, connection_RAY)

CPU times: total: 93.8 ms
Wall time: 1.72 s


### Save data

In [6]:
equity_template_data = equity_template['data']
for index, row in d.iterrows():
    for j in range(1, 10):
        equity_template_data.cell(row = index+2, column = j).value = row[j-1]

### Save file

In [7]:
ex_folder = "Export/"
file_name = "Equity_Uncons_" + rd + ".xlsx"
equity_template.save(ex_folder + file_name)

### Send the email 

In [8]:
outlook = win32.Dispatch('outlook.application')
mail = outlook.CreateItem(0)
mail.To = 'sebastian.almeida@rbinternational.com'
mail.Cc = ''
mail.Subject = 'Equity Data ' + rd
mail.Body = '''Dear Sebastian,

Please find attached, the Equity data for ''' + rd +'''.
The file will also be uploaded on our shared folder: 
https://rbinternational.sharepoint.com/:f:/r/sites/OCR/Shared%20Documents/OCR%20-%20Production/04%20RWA%20PROCESS/042%20MONTHLY%20LOADS/SSK%20-%20Regular%20Tasks/Equity%20Report%20-%20Sebastian%20Almeida?csf=1&web=1&e=foHqOD

Kr
Arber Bajraktari
Student Support IT

Raiffeisen Bank International AG | Am Stadtpark 9, 1030 Vienna, Austria | Tel. : +43 664 99481391
arber.bajraktari@rbinternational.com | http://www.rbinternational.com 
'''

# To attach a file to the email (optional):
attachment  = directory + ex_folder + file_name
mail.Attachments.Add(attachment)

mail.Display()